# Adjust beta functions of the BESSY II storage ring

**Setup:**
- Install C compiler: `apt install gcc`
- Install apace:`pip install -U apace`

For more info see https://apace.readthedocs.io/en/latest/

### Combinations of quadrupoles

Define combinations of quadrupoles to use for the fit

In [ ]:
quads_t2 = ("Q3T2", "Q4T2", "Q5T2")
quads_d2_d3 = quads_t2 + ("Q3D2", "Q4D2", "Q3D3", "Q4D3")
quads_t1_t3 = quads_d2_d3 + ("Q3P1T1", "Q3P2T1", "Q4P1T1", "Q4P2T1", "Q5P1T1", "Q5P2T1", "Q3T3", "Q4T3", "Q5T3")
quads_d1_d4 = quads_t1_t3 + ("Q3D1", "Q4D1", "Q3D4", "Q4D4")

### Raise horizontal beta function to 15m

- Fit beta functions to `beta_targets` while minizing the beta beat.
- The beta beat within the interval of `mask` is ignored.

In [ ]:
import apace as ap
from apace.plot import twiss_plot
import numpy as np
from scipy.optimize import minimize

In [ ]:
# OPTIONS
url = "https://raw.githubusercontent.com/NoBeam/lattices/master/b2_stduser_2019_05_07.json"
fit_element_names = quads_d2_d3
mask = 40.5, 49.5
# mask = 35, 55
beta_targets = [(45, 15, 'x'),]  # position, value, plane
method = "Nelder-Mead"
n_iterations = 10
n_points = 1000
attributes = ["k1"] * len(fit_element_names)
max_value = 3

# SETUP
bessy2_fit = ap.Lattice.from_file(url)
twiss_fit = ap.Twiss(bessy2_fit)
bessy2_ref = ap.Lattice.from_file(url)
twiss_ref = ap.Twiss(bessy2_ref)

fit_elements = [bessy2_fit[name] for name in fit_element_names]
linspace = np.linspace(0, bessy2_fit.length, n_points)
positions = np.array([x for x in linspace if not (mask[0] < x < mask[1])])
ref_beta_x = np.interp(positions, twiss_ref.s, twiss_ref.beta_x)
ref_beta_y = np.interp(positions, twiss_ref.s, twiss_ref.beta_y)
targets_indices = [np.searchsorted(twiss_ref.s, target[0]) for target in beta_targets]

# FITNESS FUNCTION
def fitness_function(params):
    for param, element, attribute in zip(params, fit_elements, attributes):
        value = max(min(param, max_value), -max_value)
        setattr(element, attribute, value)

    if not twiss_fit.stable:
        return float("inf")

    beta_x = np.interp(positions, twiss_fit.s, twiss_fit.beta_x)
    beta_y = np.interp(positions, twiss_fit.s, twiss_fit.beta_y)
    beta_x_beat = beta_x / ref_beta_x
    beta_y_beat = beta_y / ref_beta_y
    beta_x_beat[beta_x_beat < 1] = 1
    beta_y_beat[beta_y_beat < 1] = 1
    beta_x_beat **= 4
    beta_y_beat **= 4

    target_sum = 0
    for index, (pos, target_value, plane) in zip(targets_indices, beta_targets):
        beta = twiss_fit.beta_x if plane == 'x' else twiss_fit.beta_y
        current_value = beta[index]
        target_sum += abs(beta[index - 1] - beta[index + 1]) / 5 # force symmetry
        target_sum += (i + 1) / n_iterations * abs(current_value - target_value) ** 4 / target_value

    return np.mean([beta_x_beat, beta_y_beat]) + target_sum

# MINIMIZE
for i in range(n_iterations):
    start_values = np.array([getattr(*item) for item in zip(fit_elements, attributes)])
    if i == 0: 
        initial_values = start_values

    res = minimize(fitness_function, start_values, method=method)
    print(f'iteration: {i + 1}/{n_iterations}, result: {res.fun:.3f}')

# OUTPUT RESULTS
for element, attribute, initial_value in zip(fit_elements, attributes, initial_values):
    print(f"{element.name}: {attribute} = {getattr(element, attribute):.3f} ({initial_value:.3f})")
print("\n")

# PLOT
_ = twiss_plot(twiss_fit, ref_twiss=twiss_ref, sections=(30,60), fig_size=(9,5))

### Multiknob between `fit_lattice` and `ref_lattice`

In [ ]:
bessy2_int = ap.Lattice.from_file(url)

from ipywidgets import interact
@interact(factor=0.0, min=0.0, max=1.0, step=0.1)
def update_twiss_plot(factor):
    for name, attribute in zip(fit_element_names, attributes):
        value_fit = getattr(bessy2_fit[name], attribute)
        value_ref = getattr(bessy2_ref[name], attribute)
        value_int = factor * value_fit + (1 - factor) * value_ref
        setattr(bessy2_int[name], attribute, value_int)

    _ = twiss_plot(ap.Twiss(bessy2_int), ref_twiss=twiss_ref, sections=(30,60), fig_size=(9,5), y_max=30)

### Export `Lattice` object as LatticeJSON

In [ ]:
bessy2_int.as_dict()